# I attempt to demonstrate a functioning example of SMC with `coddiwomple` in the `OpenMM` regime

In [ ]:
from openmmtools import testsystems
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion
from coddiwomple.openmm_adapters import *
from coddiwomple.particles import Particle
from simtk import unit
import copy
import numpy as np

In [ ]:
#the target factory:
ala_vac_sys = testsystems.AlanineDipeptideVacuum().system
factory = AbsoluteAlchemicalFactory(consistent_exceptions = False)
alchemical_region = AlchemicalRegion(alchemical_atoms=range(22))
alanine_alchemical_system = factory.create_alchemical_system(reference_system=ala_vac_sys,
                                                            alchemical_regions = alchemical_region)



## the first thing we will do is to generate a pdf file holding all of the positions of the model at the prior distribution

In [ ]:
#generate openmm pdf states...
proposal_pdf_state = OpenMMPDFState(system = alanine_alchemical_system, pressure = None)

In [ ]:
#build a target/proposal factory
propagator = OpenMMLangevinDynamicsSplittingMove(n_steps = 1)
elecs = np.append(np.linspace(1.0, 0.0, 50), np.zeros(50))
sterics = np.linspace(1.0, 0.0, 100)
parameter_sequence = [{'lambda_electrostatics': i, 'lambda_sterics': j} for i,j in zip(elecs, sterics)]
proposal_factory = OpenMMProposalFactory(proposal_pdf_state, parameter_sequence, propagator)

In [ ]:
import os
os.system(f"rm -r test")
reporter = OpenMMReporter(trajectory_directory = 'test',
                 trajectory_prefix = 'alanine',
                 topology = testsystems.AlanineDipeptideVacuum().topology,
                 subset_indices = None)

In [ ]:
particle = Particle(0)
particle._state = OpenMMParticleState(positions = testsystems.AlanineDipeptideVacuum().positions)
particle.update_work(0.)

In [ ]:
particle.incremental_works

In [ ]:
mod_particle, heat = proposal_factory.propagate(particle)

In [ ]:
import tqdm
for q in tqdm.trange(500):
    mod_particle, heat = proposal_factory.propagate(particle)
    if q == range(100)[-1]:
        reporter.record([particle], save_to_disk = True)
    else:
        reporter.record([particle], save_to_disk = True)

In [ ]:
vars(reporter)